In [1]:
import pandas as pd
import subprocess
from tqdm import tqdm

### Video

In [2]:
vid = pd.read_csv("google2022_video_info.csv")

In [3]:
vid.head()

,filename,filepath,checksum_sha256
0,nIAa2wx0y2Q.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,3eb256475d44d124d8d671b15460066e767a87efaea137...
1,ZL_4kcX7kIM.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,16270b71732f94fd4a9bc2169f76cf94010d4e0bacbe1d...
2,QDt2eixQ-7E.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,2f5617ad11a6c8900a106261d9e56d5eb34c750c159312...
3,wAE3Adxg8z4.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,bc61eec68d979c23eacdee58c5df66a915c669d93283e8...
4,31E4r62s34c.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,d34b9f98fd5a1084181969b3ee61ebcd9f40bc79b2593f...


In [4]:
vid.shape

(25932, 3)

There is only one duplicate video. For convenience, we copy all videos to the ./data/mp4

In [5]:
vid.loc[vid.duplicated(subset =['checksum_sha256'], keep=False)]

,filename,filepath,checksum_sha256
8838,c4BdVaD2IhM.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,1efed8286640916debd33876bbf358043e0ade4938d2c4...
16155,OLrW4v7tdYU.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,1efed8286640916debd33876bbf358043e0ade4938d2c4...


In [6]:
len(vid['checksum_sha256'].unique())

25931

In [7]:
len(vid['filename'].unique())

25932

In [8]:
vid.shape

(25932, 3)

### Copy files to video processing folder

In [43]:
# for i in tqdm(vid['filepath'][:]):
#     cmd = "cp "+i+" /home/jyao01/github/google_2022/data/mp4"
#     #print(cmd)
#     subprocess.run(cmd, shell=True)

100%|█████████████████████████████████████| 25932/25932 [53:38<00:00,  8.06it/s]


Load ad id associated with youtube video id

Below is a query Pavel used for the export from MySQL on wesmedia1:

-Note from pavel: 
I'm using a WHERE clause because there were some videos for which the youtube id came in empty:\
Google has removed the videos from the ads. 
And there is a small number of videos (about 100) that failed to download.

In [ ]:
# select url, ad_id, youtube_id
#       from ad_media.video_info where char_length(youtube_id) > 0;

In [40]:
wt = pd.read_csv("ad_id_and_youtube_id.csv")

In [41]:
wt.shape

(66929, 3)

In [42]:
wt = wt.drop_duplicates(keep='last')

In [43]:
wt = wt.rename(columns={'url':'ad_url'})

In [48]:
wt['filename']=[str(i)+'.mp4' for i in wt['youtube_id']]

In [52]:
wt = wt[['ad_url', 'youtube_id', 'filename']]

In [53]:
wt.head(2)

,ad_url,youtube_id,filename
0,https://adstransparency.google.com/advertiser/...,eJ1ZsrdgNpA,eJ1ZsrdgNpA.mp4
1,https://adstransparency.google.com/advertiser/...,-MvD1AsFhSw,-MvD1AsFhSw.mp4


In [51]:
wt.ad_url[0]

'https://adstransparency.google.com/advertiser/AR03129716555194040321/creative/CR00208584571334164481?political=&region=US'

In [54]:
len(wt.ad_url.unique())

65392

In [57]:
len(wt.youtube_id.unique())

26341

In [85]:
wt.loc[wt['ad_url']=='https://adstransparency.google.com/advertiser/AR07232495815728562177/creative/CR02029189116112928769?political=&region=US']

,ad_url,youtube_id,filename


### Get video master files for the entire election period

Big query used to get the master file for the entire election circle:

In [3]:
# SELECT ad_id, ad_url, ad_type, advertiser_id, advertiser_name, date_range_start, date_range_end
# FROM `wmp-laura.g_ads_targeting.creative_stats_delta` 
# where date_range_start <= '2022-11-08'
# and date_range_end > '2021-01-06'
# and regions = 'US';

In [58]:
d = pd.read_csv("../02_masterfiles/google2022_master_01062021_11082022_US.csv")

In [59]:
d.shape

(1994681, 7)

In [60]:
d.head(2)

,ad_id,ad_url,ad_type,advertiser_id,advertiser_name,date_range_start,date_range_end
0,CR03552817975411081217,https://adstransparency.google.com/advertiser/...,TEXT,AR00053674171938373633,MIKIE SHERRILL FOR CONGRESS,2021-12-28,2022-08-16
1,CR09555900636977954817,https://adstransparency.google.com/advertiser/...,TEXT,AR00053674171938373633,MIKIE SHERRILL FOR CONGRESS,2021-02-26,2022-08-15


Double check date range - entire election cycle

In [63]:
d['date_range_start'] = pd.to_datetime(d['date_range_start'])
d['date_range_end'] = pd.to_datetime(d['date_range_end'])

In [65]:
print(d['date_range_start'].isna().sum())
print(d['date_range_end'].isna().sum())

0
0


In [77]:
df = d.loc[(d['date_range_end']> '2021-01-06')&(d['date_range_start']<='2022-11-08')]

In [78]:
df.shape

(1994681, 7)

In [79]:
df = df.merge(wt, on='ad_url', how='left', indicator=True)

In [80]:
df._merge.value_counts(dropna=True)

left_only     1294656
both           700025
right_only          0
Name: _merge, dtype: int64

In [87]:
# These youtube videos are removed by Google according to their ad policy
len(df.loc[(df['_merge']=='left_only')&(df['ad_type']=='VIDEO'), 'ad_url'].unique())

4189

In [89]:
df.ad_type.value_counts()

VIDEO    721823
IMAGE    662326
TEXT     610532
Name: ad_type, dtype: int64

In [72]:
df = df.loc[df['_merge']=='both']

In [73]:
df.head()

,ad_id,ad_url,ad_type,advertiser_id,advertiser_name,date_range_start,date_range_end,youtube_id,filename,_merge
23,CR15407070580163215361,https://adstransparency.google.com/advertiser/...,VIDEO,AR00340999924898332673,DSCC,2022-08-17,2022-08-18,tZSerBklHiI,tZSerBklHiI.mp4,both
24,CR17292530711659544577,https://adstransparency.google.com/advertiser/...,VIDEO,AR00340999924898332673,DSCC,2022-08-17,2022-08-18,sG2fOx3sexY,sG2fOx3sexY.mp4,both
31,CR01903813591904026625,https://adstransparency.google.com/advertiser/...,VIDEO,AR00536403756997148673,Fetterman for PA,2022-04-06,2022-05-17,jb_0Sdmvcf8,jb_0Sdmvcf8.mp4,both
32,CR02034014048013516801,https://adstransparency.google.com/advertiser/...,VIDEO,AR00536403756997148673,Fetterman for PA,2021-03-26,2021-03-29,EePtYDYfnGE,EePtYDYfnGE.mp4,both
35,CR03666994836252131329,https://adstransparency.google.com/advertiser/...,VIDEO,AR00536403756997148673,Fetterman for PA,2021-09-21,2021-09-22,FXnkLvHQmkg,FXnkLvHQmkg.mp4,both


In [75]:
len(df['ad_url'].unique())

62299

In [76]:
len(df['filename'].unique())

25203

In [91]:
'No digits: {:04d}, 1 digit: {:04d}, 2: {:04d}, 3: {:04d}'.format(0, 7, 42, 1514)

'No digits: 0000, 1 digit: 0007, 2: 0042, 3: 1514'